# Simulation: time series with heteroscedastic errors

## Colab environment setup

In [ ]:
!git clone https://github.com/Jayaos/TCPTS

In [ ]:
!pip install sklearn_quantile==0.0.29
!pip install scikit-learn==1.1.2
!pip install omegaconf==2.3.0

In [ ]:
%cd /content/TCPTS/
!pwd

## Load packages and data

In [ ]:
from src.SPCI import *
from src.utils import *
import matplotlib.pyplot as plt

In [ ]:
hetero_data_dict = load_data("./data/hetero_100_data_dict.pkl")

In [ ]:
plt.plot(hetero_data_dict["y"])
plt.xlabel("time point")
plt.ylabel("y")
plt.title("time series with heteroscedastic errors")
plt.plot()

## Fitting point predictor

In [ ]:
X_full = torch.Tensor(hetero_data_dict["x"])
Y_full = torch.Tensor(hetero_data_dict["y"])

In [ ]:
X_full.shape

In [ ]:
# Use 80% for training
train_frac = 0.8
N = int(X_full.shape[0] * train_frac)
X_train, X_predict, Y_train, Y_predict = X_full[:N], X_full[N:], Y_full[:N], Y_full[N:]

In [ ]:
fit_func = RandomForestRegressor(n_estimators=10, max_depth=1, criterion='squared_error',
                                bootstrap=False, n_jobs=-1, random_state=1103)
SPCI_class  = SPCI_and_EnbPI(X_train, X_predict, Y_train, Y_predict, fit_func=fit_func)
stride = 1
SPCI_class.fit_bootstrap_models_online_multistep(B = 25, fit_sigmaX=False, stride=stride)
alpha = 0.1 # 90% prediction interval

## Conformal prediction using Transformer decoder as a conditional quantile estimator

In [ ]:
SPCI_class.set_standardizer() # data standardizer

In [ ]:
# use 10% of entire data for validation and testing, respectively
SPCI_class.split_validation_test(0.5)

In [ ]:
# setting saving directory
%mkdir "../results/"
%mkdir "../results/hetero/"
SAVING_DIR = "../results/hetero/"

In [ ]:
# setting hyperparameters
STRIDE=1
PAST_WINDOW=100
# target quantiles should be given as a list where
# TARGET_QUANTILES[i] - TARGET_QUANTILES[-(i+1)] = target coverage
# in this example with i=0
# TARGET_QUANTILES[0] - TARGET_QUANTILES[-1] = 0.975 - 0.075 = 0.9 since we set target coverage to 0.9
TARGET_QUANTILES=[0.975,0.95,0.925,0.5,0.025,0.05,0.075]
DIM_MODEL=16
NUM_HEAD=4
DIM_FF=DIM_MODEL*4
NUM_LAYERS=4
DROPOUT=0.2
MAX_EPOCH=50
ADDITIONAL_TRAINING_EPOCH=0
BATCH_SIZE=4
LEARNING_RATE=0.0001
EARLY_STOP=3 # If validation loss does not improve for 3 epochs, end training
STANDARDIZE=True # To standardize data


In [ ]:
SPCI_class.compute_online_QGPT_SPCI_single_training(SAVING_DIR, STRIDE, PAST_WINDOW,
                                                    TARGET_QUANTILES, DIM_FF, NUM_HEAD, DIM_FF, NUM_LAYERS, DROPOUT,
                                                    MAX_EPOCH, ADDITIONAL_TRAINING_EPOCH,
                                                    BATCH_SIZE, LEARNING_RATE, EARLY_STOP, standardize=STANDARDIZE)

## Multi-step conformal prediction using Transformer decoder as a conditional quantile estimator

Multi-step conformal prediction requires the results (trained model, result dict, and hyperparameters dict) from the previous cell.

In [ ]:
MODEL_DIR=SAVING_DIR+"QGPT_best_model.pt"
HYPERPARAMS_DIR=SAVING_DIR+"QGPT_SPCI_hyperparms_dict.pkl"
RESULTS_DICT_DIR=SAVING_DIR+"QGPT_SPCI_result_dict.pkl"
NUM_MULTI_STEP=3

In [ ]:
SPCI_class.predict_QGPT_multistep(MODEL_DIR, HYPERPARAMS_DIR, RESULTS_DICT_DIR, NUM_MULTI_STEP)